In [78]:
## Reference: https://pythonhosted.org/scikit-fuzzy/auto_examples/plot_tipping_problem_newapi.html

In [79]:
!pip install scikit-fuzzy

In [20]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import numpy as np
from sklearn.pipeline import Pipeline
import re

In [11]:
def fuzzyModelConstruction():
  ##  For food quality
  # controller
  temperature = ctrl.Antecedent(np.linspace(0, 10, 11), 'temperature')
  flavor = ctrl.Antecedent(np.linspace(0, 10, 11), 'flavor')
  portion_size = ctrl.Antecedent(np.linspace(0, 10, 11), 'portion_size')
  attentiveness = ctrl.Antecedent(np.linspace(0, 10, 11), 'attentiveness')
  friendliness = ctrl.Antecedent(np.linspace(0, 10, 11), 'friendliness')
  speed = ctrl.Antecedent(np.linspace(0, 10, 11), 'speed')
  food_quality_tip = ctrl.Antecedent(np.linspace(0, 10, 11), 'food_quality_tip')
  service_quality_tip = ctrl.Antecedent(np.linspace(0, 10, 11), 'service_quality_tip')

  food_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'food_quality')
  service_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'service_quality')
  tip = ctrl.Consequent(np.linspace(0, 25, 26), 'tip')

  temperature.automf(3)
  flavor.automf(3)
  portion_size.automf(3)
  attentiveness.automf(3)
  friendliness.automf(3)
  speed.automf(3)
  food_quality_tip.automf(3)
  service_quality_tip.automf(3)

  food_quality['poor'] = fuzz.trimf(food_quality.universe, [0, 0, 5])
  food_quality['average'] = fuzz.trimf(food_quality.universe, [0, 5, 10])
  food_quality['good'] = fuzz.trimf(food_quality.universe, [5, 10, 10])
  service_quality['poor'] = fuzz.trimf(service_quality.universe, [0, 0, 5])
  service_quality['average'] = fuzz.trimf(service_quality.universe, [0, 5, 10])
  service_quality['good'] = fuzz.trimf(service_quality.universe, [5, 10, 10])
  tip['low'] = fuzz.trimf(tip.universe, [0, 0, 13])
  tip['medium'] = fuzz.trimf(tip.universe, [0, 13, 25])
  tip['high'] = fuzz.trimf(tip.universe, [13, 25, 25])

  # Fuzzy Rules
  rule1_food = ctrl.Rule(temperature['poor'] | flavor['poor'] | portion_size['poor'], food_quality['poor'])
  rule2_food = ctrl.Rule(temperature['average'] | flavor['average'] | portion_size['average'], food_quality['average'])
  rule3_food = ctrl.Rule(temperature['good'] & flavor['good'] & portion_size['good'], food_quality['good'])
  rule1_service = ctrl.Rule(attentiveness['poor'] | friendliness['poor'] | speed['poor'], service_quality['poor'])
  rule2_service = ctrl.Rule(attentiveness['average'] | friendliness['average'] | speed['average'], service_quality['average'])
  rule3_service = ctrl.Rule(attentiveness['good'] & friendliness['good'] & speed['good'], service_quality['good'])
  rule1_tip = ctrl.Rule(food_quality_tip['poor'] | service_quality_tip['poor'], tip['low'])
  rule2_tip = ctrl.Rule(service_quality_tip['average'] | food_quality_tip['average'], tip['medium'])
  rule3_tip = ctrl.Rule(service_quality_tip['good'] & food_quality_tip['good'], tip['high'])

  # Control System Creation and Simulation
  food_ctrl = ctrl.ControlSystem([rule1_food, rule2_food, rule3_food])
  service_ctrl = ctrl.ControlSystem([rule1_service, rule2_service, rule3_service])
  tipping_ctrl = ctrl.ControlSystem([rule1_tip, rule2_tip, rule3_tip])

  food_sim = ctrl.ControlSystemSimulation(food_ctrl)
  service_sim = ctrl.ControlSystemSimulation(service_ctrl)
  tipping_sim = ctrl.ControlSystemSimulation(tipping_ctrl)

  return food_sim, service_sim, tipping_sim

In [32]:
def prompt_input():
  param_list = ['food temperature', 'food flavor', 'food portion size', 'service attentiveness', 'service friendliness', 'service speed']
  input_params = []
  global exit_flag
  for i in range(len(param_list)):
    while True:
      user_input = input(f"Satisfaction on {param_list[i]} (0~10 | 'exit' to end): ")

      # Check if the user wants to exit
      if user_input.lower() == 'exit':
        print("\nThe program terminates.")
        exit_flag = True
        break
      # check if legal (is a number and [1,10])
      elif not re.compile(r'^[-+]?\d*\.?\d+$').match(user_input):
        print(f"\nInvalid input, please enter again")
        print(f"The input input_params are currently: {input_params}\n")
        continue
      elif (float(user_input) > 10.0 or float(user_input) < 0.0):
        print(f"\nInvalid input, {user_input} out of range, please enter again")
        print(f"The input input_params are currently: {input_params}\n")
        continue

      # Process the user input
      input_params.append(float(user_input))
      break
    if exit_flag:
      break

  # if parameters is not enough, make it zeros to prevent out of range error
  lacks = len(param_list)-len(input_params)
  if (lacks > 0):
    for j in range(lacks):
      input_params.append(0)

  return input_params

In [13]:
def pipeline(input_params):
  food_sim, service_sim, tipping_sim = fuzzyModelConstruction()

  food_sim.input['temperature'] = input_params[0]
  food_sim.input['flavor'] = input_params[1]
  food_sim.input['portion_size'] = input_params[2]
  service_sim.input['attentiveness'] = input_params[3]
  service_sim.input['friendliness'] = input_params[4]
  service_sim.input['speed'] = input_params[5]
  food_sim.compute()
  service_sim.compute()

  food_quality_output = food_sim.output['food_quality']
  service_quality_output = service_sim.output['service_quality']

  tipping_sim.input['food_quality_tip'] = food_quality_output
  tipping_sim.input['service_quality_tip'] = service_quality_output
  tipping_sim.compute()

  tip_output = tipping_sim.output['tip']

  print(f"\nThe suggested tip is {tip_output:.2f}%")

In [33]:
global exit_flag
exit_flag = False

input_params = prompt_input()
if not exit_flag:
  pipeline(input_params)

Satisfaction on food temperature (0~10 | 'exit' to end): 1
Satisfaction on food flavor (0~10 | 'exit' to end): exit

The program terminates.
